### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scraping is to automate the collection of targeted information on the web.


In [2]:
from bs4 import BeautifulSoup
import requests

Let's say I want to scrape the description of the latest movies released in theaters.

So we'll go to the Allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [3]:
url = "http://www.allocine.fr/"
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time you will see the part of the HTML script highlighted that corresponds to the area that interests you.

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [4]:
for p in soup.find_all("a"):
    print(p.text)

Meilleurs films d'animation Disney
Meilleurs films Pixar

            Cinéma
        

            Séries
        

            Streaming
        

            Trailers
        

            VOD
        

            Les indés
        
séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces
Agenda des sorties
Kids
Meilleurs films
Meilleurs documentaires
Avant-premières
Box Office
Films pour enfants à l'affiche
DVD
Meilleurs films 2019
Tous les films pour enfants
Tous les films
Sorties de la semaine
Moon Le Panda
Doux Jésus
Dog Man
The Amateur
Piégé
Mikado
Bergers
Opéra de Paris : La Belle au bois dormant (2025)
The Chosen : Dernière Cène
Zion
Ma Mère, Dieu et Sylvie Vartan
Blanche Neige
Minecraft, Le Film
Les Bodin’s partent en vrille
Tous les films encore à l'affiche
 Dragons 
 Sinners 
 La Réparation 
 Le Mélange des genres 
 Rapide 
Tous les films à venir
Dernières news cinéma
La Légende d’Ochi : un conte fabuleux avec des acteurs nommés aux Goldens Globes
"Faire u

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [5]:
# In addition to the tag `a`, which is easily identifiable, we notice some additional
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=325691.html" title="Moon Le Panda">Moon Le Panda</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000003827.html" title="Doux Jésus">Doux Jésus</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=296648.html" title="Dog Man">Dog Man</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=313550.html" title="The Amateur">The Amateur</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=313672.html" title="Piégé">Piégé</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=313954.html" title="Mikado">Mikado</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000000424.html" title="Bergers">Bergers</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000016713.html" title="Opéra de Paris : La Belle au bois dormant (2025)">Opéra de Paris : La Belle au bois dormant (2025)</a

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [6]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("href"))

/film/fichefilm_gen_cfilm=325691.html
/film/fichefilm_gen_cfilm=1000003827.html
/film/fichefilm_gen_cfilm=296648.html
/film/fichefilm_gen_cfilm=313550.html
/film/fichefilm_gen_cfilm=313672.html
/film/fichefilm_gen_cfilm=313954.html
/film/fichefilm_gen_cfilm=1000000424.html
/film/fichefilm_gen_cfilm=1000016713.html
/film/fichefilm_gen_cfilm=1000015911.html
/film/fichefilm_gen_cfilm=317103.html
/film/fichefilm_gen_cfilm=316984.html
/film/fichefilm_gen_cfilm=251421.html
/film/fichefilm_gen_cfilm=227463.html
/film/fichefilm_gen_cfilm=326019.html
/series/ficheserie_gen_cserie=27480.html
/series/ficheserie_gen_cserie=36351.html
/series/ficheserie_gen_cserie=32553.html
/series/ficheserie_gen_cserie=35476.html
/series/ficheserie_gen_cserie=27910.html
/series/ficheserie_gen_cserie=28613.html
/series/ficheserie_gen_cserie=30830.html


Can you retrieve the titles for me by searching for `"title"` in the items of the previous list?

In [7]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("title"))

Moon Le Panda
Doux Jésus
Dog Man
The Amateur
Piégé
Mikado
Bergers
Opéra de Paris : La Belle au bois dormant (2025)
The Chosen : Dernière Cène
Zion
Ma Mère, Dieu et Sylvie Vartan
Blanche Neige
Minecraft, Le Film
Les Bodin’s partent en vrille
The White Lotus
Adolescence
Good American Family
1944
Alien: Earth
Astérix & Obélix : le Combat des Chefs
L'Éternaute


#### Get summary

Let's start by building the URL that we will use to retrieve the summaries.

Start by putting the `href` values in a list of links.


In [8]:
links = []
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    links.append(elem.get("href"))
links

['/film/fichefilm_gen_cfilm=325691.html',
 '/film/fichefilm_gen_cfilm=1000003827.html',
 '/film/fichefilm_gen_cfilm=296648.html',
 '/film/fichefilm_gen_cfilm=313550.html',
 '/film/fichefilm_gen_cfilm=313672.html',
 '/film/fichefilm_gen_cfilm=313954.html',
 '/film/fichefilm_gen_cfilm=1000000424.html',
 '/film/fichefilm_gen_cfilm=1000016713.html',
 '/film/fichefilm_gen_cfilm=1000015911.html',
 '/film/fichefilm_gen_cfilm=317103.html',
 '/film/fichefilm_gen_cfilm=316984.html',
 '/film/fichefilm_gen_cfilm=251421.html',
 '/film/fichefilm_gen_cfilm=227463.html',
 '/film/fichefilm_gen_cfilm=326019.html',
 '/series/ficheserie_gen_cserie=27480.html',
 '/series/ficheserie_gen_cserie=36351.html',
 '/series/ficheserie_gen_cserie=32553.html',
 '/series/ficheserie_gen_cserie=35476.html',
 '/series/ficheserie_gen_cserie=27910.html',
 '/series/ficheserie_gen_cserie=28613.html',
 '/series/ficheserie_gen_cserie=30830.html']

The absolute URL of the movie pages we are looking for are built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to take the previous list and build the absolute URLs for our search
It's up to you to play.

NB: Do not take the links for the shows(series). We only want movies.

In [9]:
movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
movie_links

['http://www.allocine.fr/film/fichefilm_gen_cfilm=325691.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000003827.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=296648.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=313550.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=313672.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=313954.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000000424.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000016713.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000015911.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=317103.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=316984.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=251421.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=227463.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=326019.html']

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list.

In [10]:
url = movie_links[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "html")
soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=325691.html 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1,max-image-preview:standard" name="robots"/>
<title>Moon Le Panda - Film 2025 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Moon Le Panda est un film réalisé par Gilles de Maistre avec Noé Liu Martane, Sylvia Chang. Synopsis : Tian a 12 ans quand il est envoyé chez sa grand-mère à cause de ses mauvais résultats à l'écol

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<section class="section ovw ovw-synopsis" id="synopsis-details">

      <div class="content-txt">
 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
      </div>
    
      </section>
```

In [11]:
for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-lg"}):
    # Just like that
    print(elem.text)

# Get the synopsis section
for elem in soup.find_all("section", attrs={"id": "synopsis-details"}):
    # Get the text of the synopsis
    for elem2 in elem.find_all("div", attrs={"class":"content-txt"}):
        # Just like that
        print(elem2.text)


Tian a 12 ans quand il est envoyé chez sa grand-mère à cause de ses mauvais résultats à l'école. Loin de la ville, dans les mystérieuses montagnes chinoises, il se lie d’amitié en secret avec un panda qu’il nomme Moon. C’est le début d’une incroyable aventure qui va changer à tout jamais sa vie et celle de sa famille.



1) Automate this script for the entire list of movies.

2) Put the information in three lists (`film_links`, `title`, `synopsis`).

3) Create a `DataFrame` object from the `pandas` library that you will have to install. This dataframe will have to include these three informations in three columns.

4) Save this dataframe in a CSV file.

And here's your first real scrap, you're real hackers now.

In [15]:
import time
import random
from random import randint

title = []
synopsis = []

# Here are the things you will have to do for all links:
# - Slow down the frequency of requests to avoid being identified and therefore banned from the website.
# Use `time.sleep(random.uniform(1.0, 2.0))`
# - Get request object from URL
# - Extract the content into a variable using BeautifulSoup
# - Get title
# - Get synopsis

for link in movie_links:

    try:
        #slow down 
        time.sleep(random.uniform(1,3))

        #récupérer la page
        r = requests.get(link)
        soup = BeautifulSoup(r.content, "html.parser")

        #Extraction du titre
        title_tag = soup.find("div", class_="titlebar-title titlebar-title-lg")
        if title_tag:
            title.append(title_tag.text.strip())
        else:
            title.append("Titre inconnu")
        
        #Extraction du synopsis
        synopsis_section = soup.find("section", id="synopsis-details")
        if synopsis_section:
            syn = synopsis_section.get_text(strip=True)
            synopsis.append(syn)
        else:
            synopsis.append("Synopsis non trouvé")
        
    except Exception as e:
        print(f"Erreur avec le lien: {link}\n{e}")
        title.append("Erreur")
        synopsis.append("Erreur")
    

# Check the length of the lists before creating the dataframe
len(title), len(synopsis), len(movie_links)

(14, 14, 14)

In [22]:
import pandas as pd

df = pd.DataFrame({"Title": title})
df["Synopsis"] = synopsis
df["Links"] = movie_links

df.head()

,Title,Synopsis,Links
0,Titre inconnu,SynopsisTout publicTian a 12 ans quand il est ...,http://www.allocine.fr/film/fichefilm_gen_cfil...
1,Titre inconnu,"SynopsisTout publicSœur Lucie, religieuse dévo...",http://www.allocine.fr/film/fichefilm_gen_cfil...
2,Titre inconnu,SynopsisTout publicLorsqu’un policier et son f...,http://www.allocine.fr/film/fichefilm_gen_cfil...
3,Titre inconnu,"SynopsisTout publicCharlie Heller, un cryptogr...",http://www.allocine.fr/film/fichefilm_gen_cfil...
4,Titre inconnu,SynopsisInterdit - 12 ans avec avertissementUn...,http://www.allocine.fr/film/fichefilm_gen_cfil...


In [23]:
df.to_csv("./assets/allo_cine.csv", index=False)